### MY470 Computer Programming

### Problem Set 2, AT 2023

#### \*\*\* Due 12:00 noon on Monday, October 23 \*\*\*

---
### Writing your own k-means clustering algorithm

K-means clustering is a simple unsupervised machine-learning method for cluster analysis. The aim of the method is to partition a set of points into k clusters, such that each point is assigned to the nearest cluster. The algorithm iterates through two steps:

1. Assign each data point to the cluster with the nearest centroid
2. Update the centroids of the clusters given the new assignment

The algorithm converges when the assignments no longer change. Since the intial assignment to clusters is largely random, there is no guarantee that the optimum assignment is found. So it is common to run the algorithm multiple times and use different starting conditions.

In this problem set, we will implement a much simplified version of the k-means clustering algorithm. Rather than running the algorithm until convergence, we will repeat the above two steps a large but fixed number of times. In addition, we will initialize only once, using a naive method according to which we randomly choose k points from the data to use as initial cluster centroids. 

(In real life, you will of course use a library to implement such an algorithm. In Python, you can do this using [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).)

For the problem set, we will additionally use data from the file `Wholesale customers data.csv`, which you can find in the `data` repository. The file contains information on the annual spending on diverse product categories for the clients of a wholesale distributor. The data are obtained from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) and you can find more information about them [here](http://archive.ics.uci.edu/ml/datasets/Wholesale+customers#).

#### Hints

Use docstrings to describe your functions. We will subtract points from your mark if you do not use appropriate description of your code.

There are many different implementations of the k-means algorithm you can find online. However, this problem set expects you to follow the instructions and algorithms below precisely.  

In [1]:
# We will first import the modules we need
# You are expected to solve the problem set with these modules only
# Do not import and use any other ones 

# You will need the math module to estimate the square root.
# To get the square root of num, use math.sqrt(num)

import math
import csv
import random 

### Problem 1: Function to estimate Euclidean distance between two points

Write a function called `get_distance` that calculates the Euclidean distance between two n-dimensional points. The function should take two lists as arguments, where each list contains the n coordinates of each of the two points. 

Test your function for the points [0, 3, 0] and [4, 0, 0].

#### Hints

You can read about the definition of Euclidean distance on [Wikipedia](https://en.wikipedia.org/wiki/Euclidean_distance).


d(p, q) = sqrt( (p1- q1) ** 2 + (p2- q2)** 2 + (p3- q3)**2 )

In [2]:
def get_distance(point1, point2):
    """ Calculate the Euclidean distance between two n-dimensional points."""
    distance = math.sqrt(sum((x-y)**2 for x, y in zip(point1, point2)))
    return distance

dist = get_distance([0, 3, 0], [4, 0, 0])
print(f'The Euclidean distance between the give two points is {dist}.')


The Euclidean distance between the give two points is 5.0.


### Problem 2: Function to estimate the centroid of a collection of points

Write a function called `get_centroid` that estimates the centroid of a collection of n-dimensional points. The function should take one list as an argument, which contains each of the points entered as a list of n coordinates. The function should return a list with the coordinates of the virtual center point.

Test your function for the points in `test_lst` entered below.

#### Hints

The coordinate of the centroid in each dimension is the mean of the coordinates of all the points in that dimension.


In [3]:
test_lst = [[0,0,0], [0,0,1], [0,1,0], [1,0,0], 
            [0,1,1], [1,0,1], [1,1,0], [1,1,1]]

# Enter your answer to Problem 2 below. 

def get_centroid(points):
    """Calculate the centroid of a list of n-dimensional points"""
    centroid = [sum(point[i] for point in points) / len(points) for i in range(len(points[0]))] 
    return centroid


centroid = get_centroid(test_lst)
print(f'The centroid of the points in the list is {centroid}')

The centroid of the points in the list is [0.5, 0.5, 0.5]


---
### Problem 3: Function to read data

Write a function called `get_data` that opens the file `../data/Wholesale customers data.csv` and returns all the data in a list. 

Use the csv module to read the file. You can read how to do this [here](https://docs.python.org/3/library/csv.html). Make sure you do not include the column names in the data. 

Each element in the list you return should be a list of each customer's annual spending on fresh products, milk products, grocery products, frozen products, detergents and paper products, and delicatessen products. In other words, your list should contain 440 elements (customers), each of which contains six numeric elements (amounts spent on products). The function does not need to take any arguments.

Test your function by saving the data it returns in a variable called `data`. Then print the first two elements of `data`.

In [4]:
# Enter your answer to Problem 3 here. 

def get_data():
    """ Read customer data from a CSV file and return it as a list of lists."""
    customer_data  = []
    with open('..//data/Wholesale customers data.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file) 
        next(csv_reader)  
        for row in csv_reader: 
            customer_data.append([int(i) for i in row[2:]])
    return customer_data

data = get_data()
print(data[:2])




[[12669, 9656, 7561, 214, 2674, 1338], [7057, 9810, 9568, 1762, 3293, 1776]]


---
### Problem 4: Function to implement k-means algorithm

Write a function called `kmeans` that clusters a collection of points into k clusters using a simplified version of the k-means algorithm. The function should take two arguments: 

1. `points` – a list of n-dimensional points, and
2. `k` – an integer that defines the number of desired clusters. 

The function should return two things: 

1. A clustering – a list of `k` clusters, each of which is a list of points (each of which is a list of coordinates)
2. A list of the centroids for each of the `k` clusters. Each centroid is essentially a point, so it should be presented as a list of coordinates.

Write your code around the detailed comments and the helping code below.

Test your function on the data from Problem 3 for k = 3. For each of the three clusters, print the number of customers assigned to it and the cordinates of its centroid.


In [5]:
# Enter your answer to Problem 4 in-between the code and comments below. 

def kmeans(points, k):
    """Cluster a collection of points into k clusters using k-means algorithm."""
    
    # Select k random points to use as initial centroids
    init = random.sample(points, k)

    # Create a list of k lists to contain the points assigned to each cluster.  
    clusters = [[] for i in init]
    
    # Create a list to keep the centroids of the k clusters. 
    # For now, this list will contain the points from init.
    centroids = [i for i in init]
    
    # You now need to assign each point to the cluster with the closest centroid. Use the get_distance function you wrote in Problem 1 for this.
    
    # You should then update the variable "clusters" to be the new clustering and 
    # update the variable "centroids" to contain the centroids of the clusters in this new clustering.
    # Use the function you wrote in Problem 2 to estimate the centroids.
    
    # Repeat the process described above for 100 iterations. 
    # The idea is that each new repetition refines the clustering because it starts from the centroids of the previous clustering. 
    # If we repeat the process long enough, the assignment to clusters and the centroids will become stable.

    for j in range(100):
        for point in points:
            distances = [get_distance(point, centroid) for centroid in centroids]
            nearest_cluster = distances.index(min(distances))
            clusters[nearest_cluster].append(point)

        for i in range(k):
            centroids[i] = get_centroid(clusters[i])
    
    return clusters, centroids

(clusters, centroids)= kmeans(data, 3) 


for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {len(cluster)} customers")
    print(f"Centroid: {centroids[i]}")




Cluster 1: 6099 customers
Centroid: [7244.500081980653, 16701.005902606983, 25434.315461551076, 1871.333005410723, 11357.149368748975, 2091.418429250697]
Cluster 2: 9593 customers
Centroid: [29787.04138434275, 4886.54101949338, 5786.386636088815, 5577.1986865422705, 1110.5585322631084, 2540.355154800375]
Cluster 3: 28308 customers
Centroid: [6997.379044792991, 3755.1108167302527, 4918.168115020489, 2481.6190829447505, 1655.5340539776741, 1058.6799491309878]


---

### Evaluation

| Problem | Mark     | Comment   
|:-------:|:--------:|:----------------------
| 1       |   2/2    | Good             
| 2       |   2/2    | Good
| 3       |   2/2    | Good
| 4       |   4/6    | You need to create a new clustering for each loop run, at present you repeatedly add points to the same clustering.
| Legibility      |   2/2    | Good
| Modularity      |   2/2    | Good
| Efficiency      |   2/4    | 4 - `.index(min(_))` is a hidden inefficient loop - the operation could be included within the previous loop. It is faster to iterate through items rather than indexes when updating centroids.
|**Total**|**16/20**  | Very good